# Introduction to LLM Agents with LangChain

Welcome to the workshop on building LLM agents with LangChain!

**The goal** 

With this notebook you will familiarize yourself with the key concepts of an LLM agent. At the end, you will have all the code you need for your very own agent and you will be able to build custom tools for your own use-case. 

🌟 So ... let us begin!  

**The use case** 

Summer holidays are coming up and you still don't know where to go. Oh no! 

You decide to build a tool that helps you get information on holiday locations. For example, you would like to to find out how big a specific city is, what sights are there to see, how the weather there is, and you would like to get a drawing of that place, to get a first impression. Because who does not like art? 

You will implement this through a sentinent LLM agent, who has access to
* the wikipedia API,
* a weather API,
* can generate images. 

**The steps**

0. [Install dependencies to get the project up and running](#0)
1. [Familiarize yourself with the default wikipedia tool](#1)
        <ol type="a">
        <li>[Explore tool parameters](#1a)</li>
        <li>[Run tool and explore output](#1b)</li>
        </ol>
2. [Build your own tool: weather api](#2)
3. Agent exercises

# Install dependencies <a id='0'></a>

Compile the cell below to install the dependencies. Consider clearing the cell output so it does not clutter your notebook.

In [ ]:
!python3 -m venv venv         
!source venv/bin/activate     
!pip3 install -r helper_functions/requirements.txt

## A Langchain default tool: the [Wikipedia tool](https://python.langchain.com/v0.1/docs/integrations/tools/wikipedia/) <a id='1'></a>

The cell below loads the full wikipedia tool. It makes an API call to Wikipedia using the ``WikipediaAPIWrapper`` and returns a summary of the queried article. ``WikipediaQueryRun`` then wraps this into a ready made tool.

In [6]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1)#, doc_content_chars_max=1000)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

/Users/maria/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


#### Exercise 1 (a): Familiarize yourself with a default tool <a id='1a'></a>

Use the methods ``name``, ``description``, ``args``, ``return_direct``, ``metadata`` to familiarize yourself with the parameters of the tool. What is the meaning of the different parameters?

Background: each tool is a ``BaseTool`` class object, you can find its definition [here](https://api.python.langchain.com/en/latest/tools/langchain_core.tools.BaseTool.html#langchain_core.tools.BaseTool).

In [7]:
print("Name: ", wiki_tool.name)

### enter your code here

Name:  wikipedia


#### Exercise 1 (b) run tool and explore output <a id='1b'></a>

* Use the ``.run(tool_input)`` method to execute the tool. The ``tool_input`` is the search term that you'd like to query wikipedia with.
* [Optional] Check out the arguments of the WikipediaAPIWrapper [here](https://api.python.langchain.com/en/latest/utilities/langchain_community.utilities.wikipedia.WikipediaAPIWrapper.html) and modify its parameters above. How does the output change? 

In [8]:
# enter your code here

## Custom tools

You can build your own tools and don't have to rely on default tools. Tools can be built from any function with the LangChain class method ``StructuredTool.from_function()``(see [here](https://python.langchain.com/v0.1/docs/modules/tools/custom_tools/#structuredtool-dataclass)). The basic elements are
* The **function** you would like to be executed when the tool is called
* The definition of the **input parameters**
* The tool **description**

The tool description is especially important, since this is what the agent will use to make the deicion if this tool should be used.

Below you see the wikipedia tool, built from the basic elements described above:

In [9]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import StructuredTool

# define the function
def wikipedia_caller(query:str) ->str:
    """This function queries wikipedia through a search query."""
    return api_wrapper.run(query)

# Input parameter definition
class QueryInput(BaseModel):
    query: str = Field(description="Input search query")

# the tool description
description: str = (
        "A wrapper around Wikipedia. "
        "Useful for when you need to answer general questions about "
        "people, places, companies, facts, historical events, or other subjects. "
        "Input should be a search query."
    )


# fuse the function, input parameters and description into a tool. 
my_own_wiki_tool = StructuredTool.from_function(
    func=wikipedia_caller,
    name="wikipedia",
    description=description,
    args_schema=QueryInput,
    return_direct=False,
)

# test the output of the tool
print(my_own_wiki_tool.run('pyladies'))

Page: PyLadies
Summary: PyLadies is an international mentorship group which focuses on helping more women become active participants in the Python open-source community. It is part of the Python Software Foundation. It was started in Los Angeles in 2011. The mission of the group is to create a diverse Python community through outreach, education, conferences and social gatherings. PyLadies also provides funding for women to attend open source conferences. The aim of PyLadies is increasing the participation of women in computing. PyLadies became a multi-chapter organization with the founding of the Washington, D.C., chapter in August 2011.


#### Exercise 2: Build your own tool: weather api <a id='2'></a>
The goal is to build a tool that extracts weather information from the weather site visualcrossing.com. You typically need an API key to extract information from a website. In this example we provide you with the API key. 

Task: 
- The tool function is already provided to your. Build the tool by defining the input parameters and the descriptions. 
- Turn function, description and input parameters into a tool through ``StructuredTool.from_function()``
- test if the tool gives an output

In [ ]:
import requests # for API calls

# define the function
def extract_city_weather(city:str)->str:
    api_key = 'ZHK4HRAZ3MTNV5FDNQKFYUGL7'

    # Build the API URL
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{city}?key={api_key}&unitGroup=metric"

    response = requests.get(url)

    # extract response
    if response.status_code == 200:
        data = response.json()
        current_temp = data['days'][0]['temp']
        output = f"Current temperature in {city}: {current_temp}°C"
    else:
        output = f"Error: {response.status_code}"

    return output

# Input parameter definition
class WeatherInput(BaseModel):
    # insert your code here

# the tool description
description: str = (
        # insert your code here
    )

# fuse the function, input parameters and description into a tool. 
weather_tool = StructuredTool.from_function(
    # insert your code here
)

# test the output of the tool
print(weather_tool.run('Amsterdam'))

Combine the individual tools into a list. Your collection of tools is not ready to be used by an agent.

In [ ]:
tools = [my_own_wiki_tool, weather_tool]